<a href="https://colab.research.google.com/github/kisung2577/CMPE-255/blob/main/Data_Imputation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
!pip install impyute

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from impyute.imputation.cs import mice
from sklearn.preprocessing import OrdinalEncoder

In [ ]:
df = pd.read_csv('../content/kidney_disease.csv')

In [ ]:
cols_names={"bp":"blood_pressure",
          "sg":"specific_gravity",
          "al":"albumin",
          "su":"sugar",
          "rbc":"red_blood_cells",
          "pc":"pus_cell",
          "pcc":"pus_cell_clumps",
          "ba":"bacteria",
          "bgr":"blood_glucose_random",
          "bu":"blood_urea",
          "sc":"serum_creatinine",
          "sod":"sodium",
          "pot":"potassium",
          "hemo":"haemoglobin",
          "pcv":"packed_cell_volume",
          "wc":"white_blood_cell_count",
          "rc":"red_blood_cell_count",
          "htn":"hypertension",
          "dm":"diabetes_mellitus",
          "cad":"coronary_artery_disease",
          "appet":"appetite",
          "pe":"pedal_edema",
          "ane":"anemia"}

df.rename(columns=cols_names, inplace=True)

In [ ]:
df['red_blood_cell_count'] = pd.to_numeric(df['red_blood_cell_count'], errors='coerce')
df['packed_cell_volume'] = pd.to_numeric(df['packed_cell_volume'], errors='coerce')
df['white_blood_cell_count'] = pd.to_numeric(df['white_blood_cell_count'], errors='coerce')

In [ ]:
df.drop(["id"],axis=1,inplace=True)

In [ ]:
numerical_features = []
categorical_features = []

for i in df.drop('classification', axis=1).columns:
    if df[i].nunique()>7:
        numerical_features.append(i)
    else:
        categorical_features.append(i)

In [ ]:
#Replace incorrect values
df['diabetes_mellitus'] = df['diabetes_mellitus'].replace(to_replace = {'\tno':'no','\tyes':'yes',' yes':'yes'})
df['coronary_artery_disease'] = df['coronary_artery_disease'].replace(to_replace = '\tno', value='no')
df['classification'] = df['classification'].replace(to_replace = 'ckd\t', value = 'ckd')

In [ ]:
df.loc[:,categorical_features].isnull().sum().sort_values(ascending=False)

red_blood_cells            152
pus_cell                    65
sugar                       49
specific_gravity            47
albumin                     46
pus_cell_clumps              4
bacteria                     4
hypertension                 2
diabetes_mellitus            2
coronary_artery_disease      2
appetite                     1
pedal_edema                  1
anemia                       1
dtype: int64

In [ ]:
df.loc[:,numerical_features].isnull().sum().sort_values(ascending=False)

red_blood_cell_count      131
white_blood_cell_count    106
potassium                  88
sodium                     87
packed_cell_volume         71
haemoglobin                52
blood_glucose_random       44
blood_urea                 19
serum_creatinine           17
blood_pressure             12
age                         9
dtype: int64

In [ ]:
to_encode = [feat for feat in categorical_features if df[feat].dtype=='object']

In [ ]:
to_encode

['red_blood_cells',
 'pus_cell',
 'pus_cell_clumps',
 'bacteria',
 'hypertension',
 'diabetes_mellitus',
 'coronary_artery_disease',
 'appetite',
 'pedal_edema',
 'anemia']

In [ ]:
ode = OrdinalEncoder(dtype = int)

In [ ]:
def encode(data):
    '''function to encode non-nan data and replace it in the original data'''
    #retains only non-null values
    nonulls = np.array(data.dropna())
    #reshapes the data for encoding
    impute_reshape = nonulls.reshape(-1,1)
    #encode date
    impute_ordinal = ode.fit_transform(impute_reshape)
    #Assign back encoded values to non-null values
    data.loc[data.notnull()] = np.squeeze(impute_ordinal)
    return data

#create a for loop to iterate through each column in the data
for columns in to_encode:
    encode(df[columns])

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-cop

In [ ]:
df.loc[:, categorical_features].head(10)

,specific_gravity,albumin,sugar,red_blood_cells,pus_cell,pus_cell_clumps,bacteria,hypertension,diabetes_mellitus,coronary_artery_disease,appetite,pedal_edema,anemia
0,1.020,1.0,0.0,NaN,1,0,0,1,1,0,0,0,0
1,1.020,4.0,0.0,NaN,1,0,0,0,0,0,0,0,0
2,1.010,2.0,3.0,1,1,0,0,0,1,0,1,0,1
3,1.005,4.0,0.0,1,0,1,0,1,0,0,1,1,1
4,1.010,2.0,0.0,1,1,0,0,0,0,0,0,0,0
5,1.015,3.0,0.0,NaN,NaN,0,0,1,1,0,0,1,0
6,1.010,0.0,0.0,NaN,1,0,0,0,0,0,0,0,0
7,1.015,2.0,4.0,1,0,0,0,0,1,0,0,1,0
8,1.015,3.0,0.0,1,0,1,0,1,1,0,0,0,1
9,1.020,2.0,0.0,0,0,1,0,1,1,0,1,0,1


In [ ]:
X = df.drop('classification', axis=1)

In [ ]:
X_train = X.loc[:300,]
X_test = X.loc[300:,]

In [ ]:
# MICE requires float values
X_train_numerical = X_train.loc[:,numerical_features].astype('float64')

In [ ]:
# Passing the numpy arrays to mice
X_train_numerical_imputed = mice(X_train_numerical.values)

In [ ]:
X_train.loc[:,numerical_features].isna().sum().sort_values(ascending=False)

red_blood_cell_count      127
white_blood_cell_count    102
potassium                  84
sodium                     83
packed_cell_volume         68
haemoglobin                47
blood_glucose_random       40
blood_urea                 15
serum_creatinine           14
blood_pressure             11
age                         9
dtype: int64

In [ ]:
X_train.loc[:,numerical_features] = X_train_numerical_imputed

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1835: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)


In [ ]:
X_train.loc[:,numerical_features].isna().sum().sort_values(ascending=False)

age                       0
blood_pressure            0
blood_glucose_random      0
blood_urea                0
serum_creatinine          0
sodium                    0
potassium                 0
haemoglobin               0
packed_cell_volume        0
white_blood_cell_count    0
red_blood_cell_count      0
dtype: int64

In [ ]:
!pip install fancyimpute

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 154 kB 9.6 MB/s 
  Created wheel for fancyimpute: filename=fancyimpute-0.7.0-py3-none-any.whl size=29899 sha256=d0ad3b3ca2d7f16fb50031d7481c4aab37c50765209a30c3fcbfe3116c5f74f8
  Stored in directory: /root/.cache/pip/wheels/e3/04/06/a1a7d89ef4e631ce6268ea2d8cde04f7290651c1ff1025ce68
  Created wheel for knnimpute: filename=knnimpute-0.1.0-py3-none-any.whl size=11353 sha256=23d71fc25d86870343ce196777299258cfa17c8e7057c65a1c76433497e16ded
  Stored in directory: /root/.cache/pip/wheels/72/21/a8/a045cacd9838abd5643f6bfa852c0796a99d6b1494760494e0
Successfully built fancyimpute knnimpute


In [ ]:
from fancyimpute import KNN

In [ ]:
imputer = KNN()

In [ ]:
X_train_imputed = pd.DataFrame(np.round(imputer.fit_transform(X_train)),columns = X_train.columns)

Imputing row 1/301 with 1 missing, elapsed time: 0.066
Imputing row 101/301 with 0 missing, elapsed time: 0.071
Imputing row 201/301 with 1 missing, elapsed time: 0.075
Imputing row 301/301 with 2 missing, elapsed time: 0.078


In [ ]:
X_train_imputed.isnull().sum()

age                        0
blood_pressure             0
specific_gravity           0
albumin                    0
sugar                      0
red_blood_cells            0
pus_cell                   0
pus_cell_clumps            0
bacteria                   0
blood_glucose_random       0
blood_urea                 0
serum_creatinine           0
sodium                     0
potassium                  0
haemoglobin                0
packed_cell_volume         0
white_blood_cell_count     0
red_blood_cell_count       0
hypertension               0
diabetes_mellitus          0
coronary_artery_disease    0
appetite                   0
pedal_edema                0
anemia                     0
dtype: int64

In [ ]:
X_train_imputed.describe().T

,count,mean,std,min,25%,50%,75%,max
age,301.0,52.837209,16.981266,2.0,45.0,55.0,65.0,90.0
blood_pressure,301.0,78.568106,14.165669,50.0,70.0,80.0,90.0,180.0
specific_gravity,301.0,1.000000,0.000000,1.0,1.0,1.0,1.0,1.0
albumin,301.0,1.375415,1.329885,0.0,0.0,1.0,2.0,5.0
sugar,301.0,0.588040,1.212321,0.0,0.0,0.0,0.0,5.0
red_blood_cells,301.0,0.730897,0.444232,0.0,0.0,1.0,1.0,1.0
pus_cell,301.0,0.710963,0.454070,0.0,0.0,1.0,1.0,1.0
pus_cell_clumps,301.0,0.139535,0.347081,0.0,0.0,0.0,0.0,1.0
bacteria,301.0,0.073090,0.260717,0.0,0.0,0.0,0.0,1.0
blood_glucose_random,301.0,162.129568,82.605689,22.0,106.0,133.0,193.0,490.0


In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler = MinMaxScaler()
scaler.fit(X_train_imputed)
X_train_scaled = scaler.transform(X_train_imputed)

In [ ]:
X_train_scaled = pd.DataFrame(data=X_train_scaled, columns = X_train.columns)

In [ ]:
X_train_scaled.describe()

,age,blood_pressure,specific_gravity,albumin,sugar,red_blood_cells,pus_cell,pus_cell_clumps,bacteria,blood_glucose_random,...,haemoglobin,packed_cell_volume,white_blood_cell_count,red_blood_cell_count,hypertension,diabetes_mellitus,coronary_artery_disease,appetite,pedal_edema,anemia
count,301.000000,301.000000,301.0,301.000000,301.000000,301.000000,301.000000,301.000000,301.000000,301.000000,...,301.000000,301.000000,301.000000,301.000000,301.000000,301.000000,301.000000,301.000000,301.000000,301.000000
mean,0.577696,0.219755,0.0,0.275083,0.117608,0.730897,0.710963,0.139535,0.073090,0.299422,...,0.608685,0.597856,0.276908,0.367663,0.495017,0.458472,0.112957,0.272425,0.252492,0.199336
std,0.192969,0.108967,0.0,0.265977,0.242464,0.444232,0.454070,0.347081,0.260717,0.176508,...,0.176276,0.177068,0.111845,0.149452,0.500808,0.499102,0.317067,0.445949,0.435165,0.400166
min,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.488636,0.153846,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.179487,...,0.500000,0.477273,0.218814,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.602273,0.230769,0.0,0.200000,0.000000,1.000000,1.000000,0.000000,0.000000,0.237179,...,0.642857,0.590909,0.280487,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.715909,0.307692,0.0,0.400000,0.000000,1.000000,1.000000,0.000000,0.000000,0.365385,...,0.714286,0.704545,0.309267,0.500000,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000
max,1.000000,1.000000,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
# MICE requires float values
X_test_numerical = X_test.loc[:,numerical_features].astype('float64')

In [ ]:
X_test_numerical_imputed = mice(X_test_numerical.values)
X_test.loc[:,numerical_features] = X_test_numerical_imputed

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1835: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)


In [ ]:
X_test_imputed = pd.DataFrame(np.round(imputer.fit_transform(X_test)),columns = X_test.columns)

Imputing row 1/100 with 2 missing, elapsed time: 0.006


In [ ]:
scaler.fit(X_test_imputed)
X_test_scaled = scaler.transform(X_test_imputed)

In [ ]:
X_test_scaled = pd.DataFrame(data=X_test_scaled, columns = X_test.columns)